In [ ]:
!pip install datasets
!pip install imblearn

In [ ]:
from datasets import load_dataset
from datasets import Dataset
from sklearn.model_selection import train_test_split
import pandas as pd

dataset_name = "OsBaran/tryTr5"  
dataset = load_dataset(dataset_name)

df = pd.DataFrame(dataset['train'])
print("Eksik veri sayısı:\n", df.isnull().sum())

df = df.dropna(subset=['text', 'label'])

print("Temizleme sonrası boyut:", df.shape)

print(df['label'].value_counts())

balanced_train_data = Dataset.from_pandas(df)
dataset_name = "OsBaran/tryTr4"  # Dataset adını uygun şekilde değiştir

dataset = load_dataset(dataset_name)

df = pd.DataFrame(dataset['train'])
# None veya NaN değerler
print("Eksik veri sayısı:\n", df.isnull().sum())

df = df.dropna(subset=['text', 'label'])

print("Temizleme sonrası boyut:", df.shape)

print(df['label'].value_counts())

test_data = Dataset.from_pandas(df)
# test_data = load_dataset("OsBaran/tryTr5")["train"]
# train_data, test_data = train_test_split(df, test_size=0, stratify=df['label'], random_state=42)

# print("Eğitim Seti Dağılımı:\n", train_data['label'].value_counts(normalize=True))
# print("Test Seti Dağılımı:\n", test_data['label'].value_counts(normalize=True))

# from imblearn.over_sampling import RandomOverSampler

# ros = RandomOverSampler(random_state=42)
# balanced_train_data, balanced_train_labels = ros.fit_resample(
#     train_data[['text']], train_data['label']
# )

# balanced_train_data = pd.DataFrame(balanced_train_data, columns=["text"])
# balanced_train_data["label"] = balanced_train_labels

# print("Dengeli Eğitim Seti Dağılımı:\n", balanced_train_data['label'].value_counts())
test_data

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
vectorizer = TfidfVectorizer(max_features=5000)  
X_train = vectorizer.fit_transform(balanced_train_data['text'])
X_test = vectorizer.transform(test_data['text'])

y_train = balanced_train_data['label']
y_test = test_data['label']
# Logistic Regression Modeli
logreg_model = LogisticRegression(random_state=42, max_iter=1000)
logreg_model.fit(X_train, y_train)
# Test Seti Tahminleri
y_pred = logreg_model.predict(X_test)

print("Classification Report:")
print(classification_report(y_test, y_pred))

# Karışıklık Matrisi
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["0 (Yanlış)", "1 (Doğru)"], yticklabels=["0 (Yanlış)", "1 (Doğru)"])
plt.xlabel("Tahmin Edilen Değer")
plt.ylabel("Gerçek Değer")
plt.title("Lojistik Regresyon Konfüzyon Matrisi")
plt.show()
kesinlik = precision_score(y_test, y_pred, average='binary')  
duyarlilik = recall_score(y_test, y_pred, average='binary')
f1_skoru = f1_score(y_test, y_pred, average='binary')
dogruluk = accuracy_score(y_test, y_pred)

print("\nMetrics:")
print(f"Kesinlik (Precision): {kesinlik:.2f}")
print(f"Duyarlılık (Recall): {duyarlilik:.2f}")
print(f"F1 Skoru: {f1_skoru:.2f}")
print(f"Doğruluk (Accuracy): {dogruluk:.2f}")
